# PyOCRのインストール

In [21]:
!pip3 install pyocr

# tesseract-ocrのインストール

In [22]:
!apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 102 not upgraded.


# tesseract-ocrのVersionを確認

4.0.0-beta.1を前提に説明していきます

In [26]:
!tesseract -v

tesseract 4.0.0-beta.1
 leptonica-1.75.3
  libgif 5.1.4 : libjpeg 8d (libjpeg-turbo 1.5.2) : libpng 1.6.34 : libtiff 4.0.9 : zlib 1.2.11 : libwebp 0.6.1 : libopenjp2 2.3.0



# 学習済みモデルのダウンロード

https://github.com/tesseract-ocr/tessdata/blob/4.0.0/ より4.0.0の学習済みモデルをダウンロード

In [27]:
!wget https://github.com/tesseract-ocr/tessdata/blob/4.0.0/jpn.traineddata?raw=true -O jpn.traineddata

--2021-08-27 04:56:56--  https://github.com/tesseract-ocr/tessdata/blob/4.0.0/jpn.traineddata?raw=true
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/tesseract-ocr/tessdata/raw/4.0.0/jpn.traineddata [following]
--2021-08-27 04:56:56--  https://github.com/tesseract-ocr/tessdata/raw/4.0.0/jpn.traineddata
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata/4.0.0/jpn.traineddata [following]
--2021-08-27 04:56:57--  https://raw.githubusercontent.com/tesseract-ocr/tessdata/4.0.0/jpn.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent

# 指定フォルダにコピー

/usr/share/tesseract-ocr/4.00/tessdata/ 以下にコピーします

In [ ]:
!cp jpn.traineddata /usr/share/tesseract-ocr/4.00/tessdata/

# PyOCRにtesseract-ocrを取り込み

In [1]:
import os
from PIL import Image
import pyocr
import pyocr.builders

ocrs = pyocr.get_available_tools()
ocr = ocrs[0]

# Testデータで認識確認

test1.png, test2.png, test3.png をつかってテストデータの認識が問題ないか確認します

In [2]:
test_data = Image.open("test1.png")
builder = pyocr.builders.TextBuilder(tesseract_layout=3)
result = ocr.image_to_string(test_data, lang="jpn", builder=builder)

print(result)

あ い う え お


# Videoカメラの認識確認

下記コマンドでVideoカメラが認識できているか確認します

In [3]:
!ls /dev/video0

/dev/video0


# Videoカメラの取り込み

本サンプルではオートフォーカス対応のカメラが必要です。<br>
奨励カメラ: Logicool C920n, Logicool C615n

In [4]:
from jetcam.usb_camera import USBCamera

#TODO change capture_device if incorrect for your system
camera = USBCamera(width=640, height=480, capture_width=640, capture_height=480, capture_device=0)

In [5]:
image = camera.read()

# カメラ画像のNotebookへの表示

In [6]:
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

image_widget = ipywidgets.Image(format='jpeg')

image_widget.value = bgr8_to_jpeg(image)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

# カメラ画像をUpdateする処理

In [14]:
camera.running = True
image = ""
def update_image(change):
    global image
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)

# OCRボタンが押されたら呼び出される処理

In [18]:
def run_ocr():
    global image
    print("start scanning:")
    ocr_image = Image.frombytes('RGB', (640,480), image)
    builder = pyocr.builders.TextBuilder(tesseract_layout=3)
    result = ocr.image_to_string(ocr_image , lang="jpn", builder=builder)
    print("result")
    print("------------------")
    print(result)
    print("------------------")

# OCRボタンの処理

In [19]:
ocr_button = ipywidgets.Button(description='ocr')
ocr_button.on_click(lambda c: run_ocr())
tool_widget = ipywidgets.VBox([
    ocr_button
])

# 画像とOCRボタンを表示

結果の表示は、JupyterLabのメニューの

In [20]:
display(tool_widget)
display(image_widget)
camera.observe(update_image, names='value')

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

# 終了処理

カメラの更新を停止

In [20]:
camera.unobserve(update_image, names='value')

# OSの初期化

カメラも初期化されます

In [ ]:
import os
os._exit(00)